In [1]:
from typing import Annotated,Sequence,TypedDict,List
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage,AIMessage,BaseMessage,SystemMessage,ToolMessage
from langgraph.graph import StateGraph,START,END
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
import os
import time
import asyncio
from lean_interact import LeanREPLConfig, AutoLeanServer, Command,LocalProject
from lean_interact.interface import CommandResponse
import nest_asyncio
from git import Repo
from utils.VectorStore import VectorStore
from typing import Optional, TypedDict

nest_asyncio.apply()
load_dotenv()

[23:51:31] INFO     For optimal use on Windows, enable long paths by running this command as            ]8;id=701854;file://d:\git_clone\LEL\.pixi\envs\default\Lib\site-packages\lean_interact\utils.py\utils.py]8;;\:]8;id=585059;file://d:\git_clone\LEL\.pixi\envs\default\Lib\site-packages\lean_interact\utils.py#93\93]8;;\
                    administrator:                                                                                 

           INFO     New-ItemProperty -Path "HKLM:\SYSTEM\CurrentControlSet\Control\FileSystem" -Name    ]8;id=619052;file://d:\git_clone\LEL\.pixi\envs\default\Lib\site-packages\lean_interact\utils.py\utils.py]8;;\:]8;id=713372;file://d:\git_clone\LEL\.pixi\envs\default\Lib\site-packages\lean_interact\utils.py#94\94]8;;\
                    LongPathsEnabled -Value 1 -PropertyType DWord -Force                                           

           INFO     For optimal use on Windows, configure git for long paths by running:               ]8;id=620344;file://d:\git_clone\LEL\.pixi\envs\default\Lib\site-packages\lean_interact\utils.py\utils.py]8;;\:]8;id=83652;file://d:\git_clone\LEL\.pixi\envs\default\Lib\site-packages\lean_interact\utils.py#112\112]8;;\

           INFO     git config --global core.longpaths true                                            ]8;id=156561;file://d:\git_clone\LEL\.pixi\envs\default\Lib\site-packages\lean_interact\utils.py\utils.py]8;;\:]8;id=974648;file://d:\git_clone\LEL\.pixi\envs\default\Lib\site-packages\lean_interact\utils.py#113\113]8;;\

True

In [5]:
vec_store=VectorStore(r".\src\src_md",r".\src\src_VectorStore")
retriever_tool=vec_store.retriever_tool
#vec_store.sync_md(r".\src\src_pdf")

local_repo_path = "./mathlib4"
if not os.path.exists(local_repo_path):
    print(f"本地路径 {local_repo_path} 不存在，开始克隆 GitHub 仓库...")
    try:
        # 使用 ! 执行系统命令克隆仓库
        github_repo_url="https://github.com/leanprover-community/mathlib4.git"
        Repo.clone_from(github_repo_url, local_repo_path)
        print(f"GitHub 仓库 {github_repo_url} 已成功克隆到 {local_repo_path}")
    except Exception as e:
        print(f"克隆仓库时出错: {e}")
else:
    print(f"本地路径 {local_repo_path} 已存在，跳过克隆操作。")



sync_store 完成
本地路径 ./mathlib4 已存在，跳过克隆操作。


In [4]:
config = LeanREPLConfig(project=LocalProject(local_repo_path))
server = AutoLeanServer(config)
server.run(Command(cmd="theorem ex (n : Nat) : n = 5 → n = 5 := id"))

CommandResponse(env=0)

In [ ]:
class NatureState(TypedDict):
    query : str | None 
    background : str | None 

class LeanReply(TypedDict):
    leanReply : dict[str, str]
    leanRunRes : dict[str, CommandResponse]


In [ ]:
vec_store.add_texts(["hello", "world"], metadatas=[{"source": "test"}])

